In [1]:
from collections import Counter, defaultdict
import math, re
from pprint import pprint

count = dict()
count_c = defaultdict(lambda: 0)
for line in open('count_1edit.txt', 'r', encoding='utf8'):
    wc, num = line.strip().split('\t')
    w, c = wc.split('|')
    count[(w, c)] = int(num)
    count_c[c] += int(num)
Ncount = Counter(count.values())

In [2]:
Nall = len(count.keys())
N0 = 26*26*26*26+2*26*26*26+26*26 - Nall
Nr = [ N0 if r == 0 else Ncount[r] for r in range(12) ]

In [3]:
def smooth(count, r=10):
    if count <= r:
        return (count+1)*Nr[count+1] / Nr[count]
    else:
        return count

def Pedit(w, c):
    if (w, c) not in count and count_c[c] > 0:
        return smooth(0) / count_c[c]
    if count_c[c] > 0:
        return smooth(count[(w, c)]) / count_c[c]
    else:
        return 0

In [4]:

def words(text): return re.findall(r'\w+', text.lower())
WORDS = Counter(words(open('big.txt').read()))
def Pw(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    states = [ ('', word, 0, Pw(word), 1) ]
    for i in range(len(word)):
        print(i, states[:3])
        STATES = [ s for state in states for s in next_states(state) ]
        states = sorted(STATES, key=lambda x: x[2])

        unique, new_states = set(), []
        for state in states:
            if state[0] + state[1] in unique: continue

            unique.add(state[0] + state[1])
            new_states.append(state)
        states = new_states
        states = sorted(states, key=lambda x: P(x[3], x[4]), reverse=True) [:500]# [:MAXBEAM]
    return states[:3]

def next_states(state):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    L, R, edit, prob, ped = state
    R0, R1 = R[0], R[1:]
    if edit == 2: return [( L + R0, R1, edit, prob, ped*0.8 )]
    noedit    = [( L + R0, R1, edit, prob, ped*0.8 )]
    delete    = [( L, R1, edit+1, Pw(L + R1), ped * Pedit(L[-1]+R0, L[-1]))]  if len(L) > 0 else []
    insert    = [( L + R0 + c, R1, edit+1, Pw(L + R0 + c + R1), ped * Pedit(R0, R0 + c) ) for c in letters]
    replace   = [( L + c, R1, edit+1, Pw(L + c + R1), ped * Pedit(R0, c) ) for c in letters]
    transpose = [( L[:-1] + R0, L[-1] + R1, edit+1, Pw(L[:-1] + R0 + L[-1] + R1), ped * Pedit(L[-1]+R0, R0+L[-1]) )] if len(L) > 1 else []
    return set(noedit + delete + replace + insert + transpose)

'''Combining channel probability with word probability to score states'''
def P(pw, pedit):
    return pw*pedit

# def spellAlign(word, correct):
#     L, R, edits = '', word, ''
#     states = [ [L, R, ''] ]
#     for i in range(len(word)+______ ):
#         states = [ state for states in map(next_states, states) for state in states ]
#         states = [ state for state in states if state[0] ___________________   )
#     states = [ state for state in states if state[0] ___________________ )
#     states.sort( ______________________________________________ )
#     return _____________________

pprint(correction('appearant'))
pprint(correction('runing'))
# pprint(correction('whou'))

0 [('', 'appearant', 0, 0.0, 1)]
1 [('a', 'ppearant', 0, 0.0, 0.8), ('as', 'ppearant', 1, 0.0, 0.8636363636363636), ('x', 'ppearant', 1, 0.0, 4.467344391085126e-06)]
2 [('ap', 'pearant', 0, 0.0, 0.6400000000000001), ('aj', 'pearant', 1, 0.0, 9.896886035634742e-06), ('apg', 'pearant', 1, 0.0, 0.0)]
3 [('app', 'earant', 0, 0.0, 0.5120000000000001), ('apc', 'earant', 1, 0.0, 0.004615984405458091), ('appt', 'earant', 1, 0.0, 0.6400000000000001)]
4 [('appe', 'arant', 0, 0.0, 0.40960000000000013), ('appek', 'arant', 1, 0.0, 0.00032936836726592424), ('appep', 'arant', 1, 0.0, 0.06357190635451507)]
5 [('appea', 'rant', 0, 0.0, 0.32768000000000014), ('appen', 'rant', 1, 0.0, 0.003413333333333334), ('appez', 'rant', 1, 0.0, 3.6596485251769365e-06)]
6 [('appear', 'ant', 0, 0.0, 0.2621440000000001), ('appeary', 'ant', 1, 0.0, 0.294071794871795), ('appeay', 'ant', 1, 0.0, 0.00469071574642127)]
7 [('appare', 'nt', 2, 3.764840868244015e-05, 0.015916815434813453), ('appeara', 'nt', 0, 0.0, 0.209715200

In [5]:
correction('thenks')

0 [('', 'thenks', 0, 0.0, 1)]
1 [('t', 'henks', 0, 0.0, 0.8), ('to', 'henks', 1, 0.0, 0.9264705882352942), ('ti', 'henks', 1, 0.0, 0.5764705882352941)]
2 [('j', 'enks', 2, 8.963906829152417e-07, 3.840111769156979e-07), ('th', 'enks', 0, 0.0, 0.6400000000000001), ('ta', 'enks', 1, 0.0, 0.0003507701692847339)]
3 [('tha', 'nks', 1, 3.495923663369443e-05, 0.18048192771084343), ('thi', 'nks', 1, 2.2409767072881046e-05, 0.21833333333333335), ('tru', 'nks', 2, 4.481953414576209e-05, 0.000978807078872624)]
4 [('than', 'ks', 1, 3.495923663369443e-05, 0.14438554216867475), ('thin', 'ks', 1, 2.2409767072881046e-05, 0.1746666666666667), ('trun', 'ks', 2, 4.481953414576209e-05, 0.0007830456630980993)]
5 [('thank', 's', 1, 3.495923663369443e-05, 0.1155084337349398), ('think', 's', 1, 2.2409767072881046e-05, 0.13973333333333335), ('thing', 's', 2, 0.0002877414092157926, 0.007231422505307857)]


[('thanks', '', 1, 3.495923663369443e-05, 0.09240674698795184),
 ('think', '', 2, 0.0004992896103837897, 0.005557575757575758),
 ('thinks', '', 1, 2.2409767072881046e-05, 0.11178666666666669)]

In [6]:
correction('speling')

0 [('', 'speling', 0, 0.0, 1)]
1 [('s', 'peling', 0, 0.0, 0.8), ('k', 'peling', 1, 0.0, 3.6550999563423756e-06), ('g', 'peling', 1, 0.0, 0.011580775911986103)]
2 [('fe', 'eling', 2, 0.0003244934272153175, 6.3832398460289395e-09), ('du', 'eling', 2, 8.963906829152417e-07, 2.445998449871704e-08), ('pe', 'eling', 2, 1.7927813658304835e-06, 1.947243888340219e-09)]
3 [('spel', 'ling', 1, 3.585562731660967e-06, 0.36571428571428577), ('spoi', 'ling', 2, 6.2747347804066925e-06, 0.18487903225806449), ('smi', 'ling', 2, 0.00014431889994935394, 0.003929150453955902)]
4 [('speak', 'ing', 2, 0.00016583227633931973, 0.012350901676879132), ('spell', 'ing', 1, 3.585562731660967e-06, 0.29257142857142865), ('spoil', 'ing', 2, 6.2747347804066925e-06, 0.1479032258064516)]
5 [('speaki', 'ng', 2, 0.00016583227633931973, 0.009880721341503307), ('spelli', 'ng', 1, 3.585562731660967e-06, 0.23405714285714294), ('spoili', 'ng', 2, 6.2747347804066925e-06, 0.11832258064516128)]
6 [('speakin', 'g', 2, 0.00016583227

[('speaking', '', 2, 0.00016583227633931973, 0.0063236616585621176),
 ('spelling', '', 1, 3.585562731660967e-06, 0.14979657142857147),
 ('spoiling', '', 2, 6.2747347804066925e-06, 0.07572645161290323)]